<a href="https://colab.research.google.com/github/Nastarii/PCB-defect-detection/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration

In [1]:
!pip install imageio

Upload the dataset from github

In [ ]:
!wget --no-check-certificate \
    "https://github.com/tangsanli5201/DeepPCB/archive/refs/heads/master.zip" \
    -O "/tmp/DeepPCB.zip"

# Libraries

---



In [2]:
import matplotlib as plt
import tensorflow as tf
import pandas as pd
import cv2 as cv
import zipfile
import imageio
import imgaug
import torch
import PIL
import os

Extract Dataset from zip

In [ ]:
zip_format = zipfile.ZipFile('/tmp/DeepPCB.zip','r')
zip_format.extractall('/tmp')
zip_format.close()